# Введение в машинное обучение для Java-разработчиков
### Практическое задание 2.  Деревья решений, случайный лес.
### Дата выдачи: 26.10.2023

### Дедлайн: 23:59MSK 08.11.2023

## О задании
В этом задании мы попытаемся разобраться в устройстве деревьев решений и ансамбля на основе деревьев. 

## Оценивание и штрафы
Каждая из задач (помечены тегом [task]) имеет определенное количество баллов (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. 

- от 4 до 8 баллов - оценка "3"
- от 8 до 14 баллов - оценка "4"
- 15 баллов - оценка "5"

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов, что автоматически ведет к несдаче курса. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в комментариях. 
В данном задании есть необязательные бонусные задания, выполнение которых добавляет баллы в карму :)

## Формат сдачи
Задания сдаются путем форка основного репозитория, коммита решения в мастер-ветку вашего форка и оповещении преподавателя о выполнении ДЗ. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

[[Укажите количество набранных баллов]]

## Часть 1. Дерево решений

- [task] Реализуйте подбор признака и порога для поиска условия разбиения выборки методом перебора (2 балла)
- [task] Реализуйте метод predict для инференса дерева на датасете (1 балл)
- [task] Обучите дерево на датасете ирисов Фишера (1 балл)
- Бонусное задание: В методе `_calculate_leaf_value` возвращайте вероятность классов
- Бонусное задание: Замените критерий Джини на энтропийный критерий
4 балла

In [35]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


class DecisionNode:
    def __init__(self, feature=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split:
            return DecisionNode(value=self._calculate_leaf_value(y))

        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None or best_threshold is None:
            return DecisionNode(value=self._calculate_leaf_value(y))

        mask = X[:, best_feature] <= best_threshold
        true_branch = self._build_tree(X[mask], y[mask], depth + 1)
        false_branch = self._build_tree(X[~mask], y[~mask], depth + 1)

        return DecisionNode(feature=best_feature, threshold=best_threshold, true_branch=true_branch,
                            false_branch=false_branch)

    def _find_best_split(self, X, y):
        best_feature = None
        best_threshold = None
        best_gain = 0
        n_features = X.shape[1]
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                cond = X[:, feature] <= threshold
                inv_cond = [not elem for elem in cond]
                gain = self._calculate_gain(y, y[cond], y[inv_cond])
                if gain > best_gain:
                    best_feature = feature
                    best_threshold = threshold
                    best_gain = gain

        return best_feature, best_threshold

    def _calculate_gain(self, y, y_true, y_false):
        p = len(y_true) / len(y)
        impurity_before = self._gini_impurity(y)
        impurity_after = p * self._gini_impurity(y_true) + (1 - p) * self._gini_impurity(y_false)
        return impurity_before - impurity_after

    def _gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _calculate_leaf_value(self, y):
        classes, counts = np.unique(y, return_counts=True)
        most_common_class = classes[np.argmax(counts)]
        return most_common_class

    def predict(self, X):
        predictions = []
        for x in X:
            predictions.append(self._predict(x, self.root))
        return np.array(predictions)

    def _predict(self, X, node):
        if node.value is not None:
            return node.value

        if X[node.feature] <= node.threshold:
            return self._predict(X, node.true_branch)
        else:
            return self._predict(X, node.false_branch)


def print_tree(node, depth=0):
    indent = "    " * depth
    if node.value is not None:
        print(indent + "Predicted value:", node.value)
    else:
        print(indent + "Feature", node.feature, "<=", node.threshold)
        print(indent + "--> True:")
        print_tree(node.true_branch, depth + 1)
        print(indent + "--> False:")
        print_tree(node.false_branch, depth + 1)

In [36]:
data = load_iris(as_frame=True)
X = data["data"].values
y = data["target"].values
### 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtree = DecisionTree()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

Accuracy: 1.0


### Часть 2. Случайный лес

##### [task] 1. Усреднение классификаторов (2 балла)
Реализуйте бэггинг над решающими деревьями (усреднение предсказанных вероятностей всего ансамбля или путем "голосования"). В качестве базового алгоритма используйте либо наш класс или DecisionTreeClassifier из пакета scikit-learn. 

##### [task] 2. Бутстрап обучающей выборки (2 балла)
Добавим к нашему усреднению предсказаний бутстрап выборки (генерация случайной выборки того же размера с возвращением). Сгенерируйте с помощью него отдельную обучающую выборку для каждого дерева, обучите их и усредните предсказания, как в предыдущем пункте.

##### [task] 3. Выбор случайного подмножества признаков при построении нового дерева. (2 балла)
Обучайте каждое дерево на отдельной бутстрап-выборке и случайно выбирайте признаки для обучения. 

##### [task] 4. Подсчитайте метрики оценки качества классификации (accuracy, precision, recall, ROC AUC, F-мера) для каждого из вариантов дерева. Сделайте отдельную функцию для подсчета метрик (1 балл)

- Используемый датасет https://archive.ics.uci.edu/dataset/94/spambase
- Все гиперпараметры необходимо вынести аргументы соответсвующей функции.

In [37]:
# fetch dataset 
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo

spambase = fetch_ucirepo(id=94)

# data (as numpy arrays) 
X = spambase.data.features.values
y = spambase.data.targets.values

# metadata 
print(spambase.metadata)

# variable information 
print(spambase.variables)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

{'uci_id': 94, 'name': 'Spambase', 'repository_url': 'https://archive.ics.uci.edu/dataset/94/spambase', 'data_url': 'https://archive.ics.uci.edu/static/public/94/data.csv', 'abstract': 'Classifying Email as Spam or Non-Spam', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 4601, 'num_features': 57, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1999, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C53G6X', 'creators': ['Mark Hopkins', 'Erik Reeber', 'George Forman', 'Jaap Suermondt'], 'intro_paper': None, 'additional_info': {'summary': 'The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...\n\nThe classification task for this dataset is to determine whether a given email is spam or not.\n\t\nOur collecti

In [39]:
from sklearn.utils import resample
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, accuracy_score

#### Your code here
class BaggingClassifier:
    def __init__(self, base_classifier, n_estimators=10):
        self.classifier_list = None
        self.base_classifier = base_classifier
        self.n_estimators = n_estimators

    def fit(self, X, y):
        self.classifier_list = []
        for _ in range(self.n_estimators):
            estimator = self.base_classifier()
            X_bootstrap, y_bootstrap = self.bootstrap(X, y)

            estimator.fit(X_bootstrap, y_bootstrap)
            self.classifier_list.append(estimator)

    def predict(self, X):
        y_pred = np.zeros((X.shape[0], len(self.classifier_list)))
        for i, estimator in enumerate(self.classifier_list):
            y_pred[:, i] = estimator.predict(X)

        return np.mean(y_pred, axis=1).astype(int)

    def bootstrap(self, X, y):
        return resample(X, y, replace=True)


def evaluate(y, y_pred):
    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    auc = roc_auc_score(y, y_pred)
    f = f1_score(y, y_pred)
    print("Accuracy:", acc)
    print("Precision:", prec)
    print("Recall:", rec)
    print("ROC AUC:", auc)
    print("F1 score: ", f)


bagging = BaggingClassifier(DecisionTree)
bagging.fit(X_train, y_train)

y_pred = bagging.predict(X_test)

evaluate(y_test, y_pred)

Accuracy: 0.8501628664495114
Precision: 0.9960629921259843
Recall: 0.6487179487179487
ROC AUC: 0.8234173547732869
F1 score:  0.7857142857142858


#### Часть 3. 

- [task] Обучите RandomForestClassifier из sklearn на датасете из прошлой части (2 балла)
- [task] Подсчитайте accuracy, precision, recall, ROC AUC, F-мера на отложенной выборке. Получилось лучше или хуже по сравнению с вашим вариантом RandomForest (2 балла)
- Попробуйте объяснить результат.

In [40]:
from sklearn.ensemble import RandomForestClassifier

#### Your code here
X = spambase.data.features.values
y = spambase.data.targets.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
evaluate(y_test, y_pred)

Accuracy: 0.9402823018458197
Precision: 0.971830985915493
Recall: 0.8846153846153846
ROC AUC: 0.9328914964508185
F1 score:  0.9261744966442953


C:\Users\dimit\miniconda3\envs\dmit_env\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Практически все показатели, за исключением Precision показали более высокий результат. Более низкие показатели собственной реализации могут быть связаны с тем, что в RandomForestClassifier используется механизм оценки важности признаков, а также с тем, что RandomForestClassifier использует голосование в качестве бэггинга.